

```
# This is formatted as code
```

# DMA Fall 21

**Note** : This entire lab will be manually evaluated.

Name : 'Tyler Freund'
Collaborator : ''

# Lab 4: Neural Networks #


In [304]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [305]:
!wget http://askoski.berkeley.edu/~zp/lab_4_training.csv
!wget http://askoski.berkeley.edu/~zp/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2021-09-22 18:18:56--  http://askoski.berkeley.edu/~zp/lab_4_training.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79177 (77K) [text/csv]
Saving to: ‘lab_4_training.csv.12’

lab_4_training.csv. 100%[===================>]  77.32K   491KB/s    in 0.2s    

2021-09-22 18:18:57 (491 KB/s) - ‘lab_4_training.csv.12’ saved [79177/79177]

--2021-09-22 18:18:57--  http://askoski.berkeley.edu/~zp/lab_4_test.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26519 (26K) [text/csv]
Saving to: ‘lab_4_test.csv.12’

lab_4_test.csv.12   100%[===================>]  25.90K  --.-KB/s    in 0.02s   

2021-09-22 18:18:57 (1.57 MB/s) - ‘lab_4_test.csv.12’ save

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0
1,677,male,19,second,hazel,72.0,120.0,1,1,16.0,Yes,9.0,70.0,3.0,5.0
2,1738,male,20,second,brown,63.0,55.0,1,2,15.0,Yes,4.5,15.0,4.0,13.0
3,1355,male,20,third,green,78.0,200.0,0,0,10.0,Yes,9.0,20.0,10.0,10.0
4,891,female,19,second,green,67.0,280.0,2,0,4.0,Yes,2.0,164.0,0.0,2.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of 'gender'. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives)

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [306]:
# YOUR CODE HERE
num_male = df_train.loc[df_train['gender'] == "male"]
num_male = len(num_male)
num_female = df_train.loc[df_train['gender'] == "female"]
num_female = len(num_female)

baseline_accuracy = num_female/(num_female+num_male)
baseline_accuracy

0.5427852348993288

###ANSWER: 0.5427852348993288

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [307]:
# YOUR CODE HERE
num_male = df_test.loc[df_test['gender'] == "male"]
num_male = len(num_male)
num_female = df_test.loc[df_test['gender'] == "female"]
num_female = len(num_female)

baseline_accuracy = num_female/(num_female+num_male)
baseline_accuracy

0.5226130653266332

###ANSWER: 0.5226130653266332


***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [308]:
# YOUR CODE HERE
df_train['gender'].replace({'female': 0, 'male': 1},inplace=True)

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=100, 
                    activation= 'logistic', solver='lbfgs', verbose=1,  random_state=1)

X1 = df_train['height'].to_numpy()
X_train = np.reshape(X1, (-1, 1))

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on training---')
y_pred_train = clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

Accuracy on training---
0.7676174496644296


###ANSWER: 0.7676174496644296

**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [309]:
# YOUR CODE HERE
df_test['gender'].replace({'female': 0, 'male': 1},inplace=True)

X2 = df_test['height'].to_numpy()
X_test = np.reshape(X2, (-1, 1))

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))



Accuracy on test---
0.7311557788944724


###ANSWER: 0.7311557788944724

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature

In [310]:
# YOUR CODE HERE
df_train['log_height'] = np.log(df_train['height'])
df_test['log_height'] = np.log(df_test['height'])

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=100,
                 solver='lbfgs', verbose=1,  random_state=1)

X1 = df_train['log_height'].to_numpy()
X_train = np.reshape(X1, (-1, 1))

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X2 = df_test['log_height'].to_numpy()
X_test = np.reshape(X2, (-1, 1))

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.8439597315436241
Accuracy on test---
0.8542713567839196


###ANSWER: 
Accuracy on train---
0.8439597315436241
Accuracy on test---
0.8542713567839196

***

### Question 3###
The rest of features in this dataset barring a few are categorical. No ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [311]:
# YOUR CODE HERE
dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.iloc[:, 13:]
dum_test = dum_test.iloc[:, 13:]

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.5713087248322147
Accuracy on test---
0.5527638190954773


###ANSWER: 
Accuracy on train---
0.5713087248322147
Accuracy on test---
0.5527638190954773

***
### Question 4###
Using a NN, report the accuracy on  the test set of a model that trained only on the height and the eye color features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original height values (no pre-processing) and eye color as a one-hot?

In [312]:
# YOUR CODE HERE
dum_test = dum_test.iloc[:, 5:10]
dum_train = dum_train.iloc[:, 5:10]

dum_train['height'] = df_train['height']
dum_test['height'] = df_test['height']

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.5696308724832215
Accuracy on test---
0.5527638190954773


###ANSWER: 
Accuracy on test---
0.5527638190954773

**Question 4.b**  
What is the accuracy on the test set using the log of height values (applied to both training and testing sets) and eye color as a one-hot?

In [314]:
# YOUR CODE HERE
dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.iloc[:, 13:]
dum_test = dum_test.iloc[:, 13:]

dum_test = dum_test.iloc[:, 5:10]
dum_train = dum_train.iloc[:, 5:10]

dum_train['height'] = df_train['height']
dum_test['height'] = df_test['height']

dum_train['height'] = np.log(df_train['height'])
dum_test['height'] = np.log(df_test['height'])

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.8322147651006712
Accuracy on test---
0.8467336683417085


###ANSWER: 
Accuracy on test---
0.8467336683417085

**Question 4.c**  
What is the accuracy on the test set using the Z-score of height values and eye color as a one-hot? 

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [315]:
# YOUR CODE HERE
dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.iloc[:, 13:]
dum_test = dum_test.iloc[:, 13:]

dum_test = dum_test.iloc[:, 5:10]
dum_train = dum_train.iloc[:, 5:10]

dum_train['height'] = df_train['height']
dum_test['height'] = df_test['height']

sd_train = np.std(dum_train['height'])
mean_train = np.mean(dum_train['height'])
dum_train['height'] = (dum_train['height'] - mean_train) / sd_train

sd_test = np.std(dum_test['height'])
mean_test = np.mean(dum_test['height'])
dum_test['height'] = (dum_test['height'] - mean_test) / sd_test

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:]

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.8473154362416108
Accuracy on test---
0.8668341708542714


###ANSWER: 
Accuracy on test---
0.8693467336683417

***
### Question 5 ###
Repeat question 4 for exercise hours + eye color

In [316]:
# YOUR CODE HERE
dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.loc[:, 'eyecolor_blue':'eyecolor_other']
dum_test = dum_test.loc[:, 'eyecolor_blue':'eyecolor_other']

dum_train['exer_hours'] = df_train['exercisehours']
dum_test['exer_hours'] = df_test['exercisehours']

#5a
clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.5855704697986577
Accuracy on test---
0.5628140703517588


In [318]:
#5b
df_train = df_train[df_train.exercisehours != 0]
df_test = df_test[df_test.exercisehours != 0]

dum_train['exer_hours'] = np.log(df_train['exercisehours'])
dum_test['exer_hours'] = np.log(df_test['exercisehours'])
dum_train = dum_train.dropna(axis=0)
dum_test = dum_test.dropna(axis=0)


clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.5946666666666667
Accuracy on test---
0.5899581589958159


In [321]:
#5c
dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.loc[:, 'eyecolor_blue':'eyecolor_other']
dum_test = dum_test.loc[:, 'eyecolor_blue':'eyecolor_other']

dum_train['exer_hours'] = df_train['exercisehours']
dum_test['exer_hours'] = df_test['exercisehours']

sd_train = np.std(dum_train['exer_hours'])
mean_train = np.mean(dum_train['exer_hours'])
dum_train['exer_hours'] = (dum_train['exer_hours'] - mean_train) / sd_train

sd_test = np.std(dum_test['exer_hours'])
mean_test = np.mean(dum_test['exer_hours'])
dum_test['exer_hours'] = (dum_test['exer_hours'] - mean_test) / sd_test

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.6013333333333334
Accuracy on test---
0.5815899581589958


###ANSWER: 
#5a:
Accuracy on test---0.5628140703517588
#5b:
Accuracy on test---0.5899581589958159
#5c:
Accuracy on test---0.5815899581589958

***
### Question 6###
Combine the features from question 3, 4, and 5 (year, eyecolor, exercise, height, exercise hours). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [323]:
# YOUR CODE HERE
df_train = df_train[df_train.exercisehours != 0]
df_test = df_test[df_test.exercisehours != 0]

dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.iloc[:, 13:]
dum_test = dum_test.iloc[:, 13:]

dum_train['height'] = df_train['height']
dum_test['height'] = df_test['height']

sd_train = np.std(dum_train['height'])
mean_train = np.mean(dum_train['height'])
dum_train['height'] = (dum_train['height'] - mean_train) / sd_train

sd_test = np.std(dum_test['height'])
mean_test = np.mean(dum_test['height'])
dum_test['height'] = (dum_test['height'] - mean_test) / sd_test

dum_train['exer_hours'] = np.log(df_train['exercisehours'])
dum_test['exer_hours'] = np.log(df_test['exercisehours'])


clf = MLPClassifier(hidden_layer_sizes=(10), activation= 'logistic',
                 solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.8826666666666667
Accuracy on test---
0.7740585774058577


###ANSWER: 
Accuracy on test---0.7740585774058577

***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [324]:
# YOUR CODE HERE
df_train = df_train[df_train.exercisehours != 0]
df_test = df_test[df_test.exercisehours != 0]

dum_train = pd.get_dummies(df_train)
dum_test = pd.get_dummies(df_test)

dum_train = dum_train.iloc[:, 13:]
dum_test = dum_test.iloc[:, 13:]

dum_train['height'] = df_train['height']
dum_test['height'] = df_test['height']

sd_train = np.std(dum_train['height'])
mean_train = np.mean(dum_train['height'])
dum_train['height'] = (dum_train['height'] - mean_train) / sd_train

sd_test = np.std(dum_test['height'])
mean_test = np.mean(dum_test['height'])
dum_test['height'] = (dum_test['height'] - mean_test) / sd_test

dum_train['exer_hours'] = np.log(df_train['exercisehours'])
dum_test['exer_hours'] = np.log(df_test['exercisehours'])


clf = MLPClassifier(hidden_layer_sizes=(1000), activation= 'logistic', alpha=.01, batch_size= 5, learning_rate= 'adaptive',
                    learning_rate_init= 10, power_t=1,  max_iter=100, beta_1= .1, beta_2=.5, solver='lbfgs', verbose=1,  random_state=1)

X_train = dum_train[0:].to_numpy()

Y_train = np.array(df_train['gender'])

clf.fit(X_train, Y_train)

print('Accuracy on train---')
y_pred_train=clf.predict(X_train)
print(accuracy_score(Y_train,y_pred_train))

X_test = dum_test[0:].to_numpy()

Y_test = np.array(df_test['gender'])

print('Accuracy on test---')
y_pred_test=clf.predict(X_test)
print(accuracy_score(Y_test,y_pred_test))

Accuracy on train---
0.8386666666666667
Accuracy on test---
0.8368200836820083


###ANSWER: 

I was able to increase my accuracy score from question 6 by 6.2% (83.6%-77.4%=6.2%). I began my optimization by reading the documentation of the sklearn MLPclassifier and tried to familiaize myself with the parameters.  Next, I did some research as to which of these parameters could possibly lead me to a higher accuracy score; more specifically, whether I needed to increase or decrease these given parameters to better my accuracy score.  I proceeded to change each parameter one by one and seeing whether my output was moving closer or further from my goal.  The most significant impact I noticed on my output was when I changed my hidden layer size from 10 to 1000.  Although, I did see other changes in output when I tweaked the other parameters, none were as substantial as the hidden layer size.